In [10]:
import torch
from torch import nn
from torch.nn import init

net=nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))#pytorch已经行默认初始化
print(net)
X=torch.rand(2,4)
Y=net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 访问模型参数

In [12]:
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [13]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [14]:
class MyModel(nn.Module):
    def __init__(self,**kwargs):
        super(MyModel,self).__init__(**kwargs)
        self.weight1=nn.Parameter(torch.rand(20,20))
        self.weight2=torch.rand(20,20)
    def forward(self,x):
        pass
n=MyModel()
for name,param in n.named_parameters():
    print(name)

weight1


In [15]:
weight_0=list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 5.5922e-02,  2.2539e-01, -4.3617e-01,  8.2294e-02],
        [ 4.0425e-01, -2.6099e-02, -2.3852e-01, -1.5272e-01],
        [ 3.3737e-01,  3.5387e-04,  1.5877e-01,  8.2639e-03]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2055,  0.3595,  0.4337,  0.2040],
        [-0.0879, -0.1538, -0.1855, -0.0872]])


## 初始化模型参数

#### 均值0，标准差0.01的正态分布初始化权重参数

In [19]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[-0.0041,  0.0129,  0.0069,  0.0041],
        [ 0.0088,  0.0125, -0.0079,  0.0125],
        [ 0.0060, -0.0021,  0.0068,  0.0002]])
2.weight tensor([[ 0.0045,  0.0041, -0.0029]])


#### 常数初始化权重参数

In [20]:
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 自定义初始化方法

In [21]:
def normal_(tensor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)

In [22]:
def init_weight(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor*=(tensor.abs()>=5).float()
        
for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight(param)
        print(name,param.data)

0.weight tensor([[ 7.5242,  0.0000, -0.0000, -9.8390],
        [-0.0000,  6.9737, -6.9502,  0.0000],
        [-9.1010,  5.8230, -9.3322,  6.2604]])
2.weight tensor([[ 0.0000, -9.3768, -0.0000]])


In [23]:
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data+=1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 共享模型参数

In [24]:
linear=nn.Linear(1,1,bias=False)
net=nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [25]:
print(id(net[0])==id(net[1]))
print(id(net[0].weight)==id(net[1].weight))

True
True


In [42]:
x=torch.ones(1,1)
y=net(x).sum()
print(y)
net[0].weight.grad.data.zero_()
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
